In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [2]:
# model, learning_rate = "AdaptableLSTM", .025
model, learning_rate = "BasicNN", .0054
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = True
insertpreds = True
noise = 0
smooth = .1
stateZeroEpochs = 0

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    stateZeroEpochs = stateZeroEpochs,
    numValFolds = 5,
    epochsToUpdateLabelMods = 10,
    data_kw={"minw": 2,
            "maxw": 29,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": False,
             "response_feature_noise": noise
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 15,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






54
214
0	 train loss: 0.1657 train acc: 37.530% test acc: 38.892%
5	 train loss: 0.1522 train acc: 51.084% test acc: 50.796%
10	 train loss: 0.1492 train acc: 49.944% test acc: 49.558%
15	 train loss: 0.1462 train acc: 51.662% test acc: 51.149%
20	 train loss: 0.1443 train acc: 51.598% test acc: 51.149%
25	 train loss: 0.1436 train acc: 51.678% test acc: 51.149%
30	 train loss: 0.1427 train acc: 51.614% test acc: 50.972%
35	 train loss: 0.1419 train acc: 51.871% test acc: 50.796%
40	 train loss: 0.1412 train acc: 51.903% test acc: 50.737%
45	 train loss: 0.1406 train acc: 51.919% test acc: 50.560%
50	 train loss: 0.1401 train acc: 51.871% test acc: 50.678%
55	 train loss: 0.1396 train acc: 51.871% test acc: 50.501%
60	 train loss: 0.1391 train acc: 51.951% test acc: 50.501%
65	 train loss: 0.1386 train acc: 51.967% test acc: 50.383%
70	 train loss: 0.1382 train acc: 51.903% test acc: 50.383%
75	 train loss: 0.1378 train acc: 51.951% test acc: 50.324%
80	 train loss: 0.1374 train acc: 5

KeyboardInterrupt: 

In [ ]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()

In [ ]:
print(e.bd.features.shape, e.bd.featureList.shape)